In [4]:
import ipywidgets
import functools
import enum
import os
import IPython
import traitlets
from ipywidgets import Box, FloatSlider

In [5]:
style_txt="""
.scwidget-answer-widget{
  border-left : 0px solid;
}

.scwidget-answer-widget--change__left{
  border-left : 10px solid;
  border-left-color: #e95420;
}
"""
style_html = IPython.display.HTML("<style>"+style_txt+"</style>")
IPython.display.display(style_html)

### 1st Approach with monkey patching widget as we discussed

In [6]:
class VisualCue:      
    def __init__(self, css_style):
        self._css_style = css_style
    
    def patch_cue_to_widget(self, widget_to_observe, trait_to_observe, wigdet_to_patch=None):
        """
            We want to put the style change around a different widget than the widget with the trait
            For example the visualizers, visualizer changes its style when answer changes
        """
        if wigdet_to_patch is None:
            wigdet_to_patch = widget_to_observe
        wigdet_to_patch._css_style = self._css_style
        wigdet_to_patch.add_class(self._css_style['base'])
        wigdet_to_patch._on_trait_change = functools.partial(VisualCue._on_trait_change, wigdet_to_patch)
        wigdet_to_patch.remove_cue = functools.partial(VisualCue.remove_cue, wigdet_to_patch)
        widget_to_observe.observe(widget_to_observe._on_trait_change, trait_to_observe)
        # needs to return widget because widget is copy not reference
        return wigdet_to_patch

    def _on_trait_change(self, change=None):
        self.add_class(self._css_style['_on_trait_change'])
        
    def remove_cue(self):
        self.remove_class(self._css_style['_on_trait_change'])
        
style = {'base': 'scwidget-answer-widget', '_on_trait_change': 'scwidget-answer-widget--change__left'}
float_slider = FloatSlider()
SAVE_CUE = VisualCue(style)
float_slider = SAVE_CUE.patch_cue_to_widget(FloatSlider(), "value")
display(float_slider)

# change value
# reload by button click


FloatSlider(value=0.0, _dom_classes=('scwidget-answer-widget',))

In [7]:
float_slider.value = 55.

In [8]:
float_slider.remove_cue()

### 2nd Approach with Box, it is same complexity, but allows nested boxes  if we want

In [17]:
class VisualCue(Box):
    def __init__(self, widget_to_observe, trait_to_observe, widget_to_frame=None, css_style=None, *args, **kwargs):
        self._css_style = css_style
        self._widget_to_frame = widget_to_observe if (widget_to_frame is None) else widget_to_frame
        self._widget_to_observe = widget_to_observe
        self._css_style = css_style
        super().__init__([self._widget_to_frame], *args, **kwargs)
        self._widget_to_observe.observe(self._on_trait_change, trait_to_observe)
        self.add_class(f"{self._css_style['base']}")
        
    def _on_trait_change(self, change=None):
        self.add_class(self._css_style['_on_trait_change'])
        
    def remove_cue(self):
        self.remove_class(self._css_style['_on_trait_change'])

float_slider2 = FloatSlider()
style = {'base': 'scwidget-answer-widget', '_on_trait_change': 'scwidget-answer-widget--change__left'}
SAVE_CUE = functools.partial(VisualCue, css_style=({'base': 'scwidget-answer-widget',
                      '_on_trait_change': 'scwidget-answer-widget--change__left'}))

float_slider_cued = SAVE_CUE(float_slider2, "value")
display(float_slider_cued)


VisualCue(children=(FloatSlider(value=0.0),), _dom_classes=('scwidget-answer-widget',))

In [18]:
float_slider2.value = 55.

In [20]:
float_slider_cued.remove_cue()